In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

sns.set_theme(style='whitegrid', font_scale=1.5)
sns.set_palette('rocket', n_colors=10)
plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)

In [17]:
import warnings
pd.options.display.max_columns = None
warnings.simplefilter(action='ignore')

### 장르별 콘서트홀 데이터 불러오기

In [18]:
symp = pd.read_csv('콘서트홀_교향곡.csv')
clas = pd.read_csv('콘서트홀_클래식.csv')
chor = pd.read_csv('콘서트홀_합창.csv')
solo = pd.read_csv('콘서트홀_독주.csv')
voca = pd.read_csv('콘서트홀_성악.csv')
cham = pd.read_csv('콘서트홀_실내악.csv')
oper = pd.read_csv('콘서트홀_오페라.csv')
conc = pd.read_csv('콘서트홀_콘서트.csv')
comp = pd.read_csv('콘서트홀_복합장르.csv')
etc = pd.read_csv('콘서트홀_기타.csv')
jazz = pd.read_csv('콘서트홀_재즈.csv')
cros = pd.read_csv('콘서트홀_크로스오버.csv')

In [19]:
conlist = [symp, clas, chor, solo, voca, cham, oper, conc, comp, etc, jazz, cros]

* * *

In [20]:
# 장르별로 카운트 다시
for c in conlist:
    c['seat_member_count'] = c.groupby('seat')['member_yn'].transform('sum')
    c['seat_paid_member_count'] = c.groupby('seat')['paid_member'].transform('sum')
    c['seat_pre_date_gap'] = c.groupby('seat')['pre_date_gap'].transform('median')

# [코로나 기간 보정]
## 좌석 count 변수 보정
**▶ 기간 : 2020.03.22 ~ 2022.01.01**
- 코로나 기간 동안 실행된 거리두기 정책으로 인해 한 칸 내지 두 칸 띄어앉기 기간 有
- 그러나 클러스터링에 활용되는 좌석 예매 횟수는 상대적인 수치이므로, **별도의 코로나 기간 보정 없이 제외하는 것**이 낫다고 판단.
- 코로나 기간의 예매 내역은 **좌석 예매 횟수 count에 반영하지 않도록** 처리 (예매 내역 자체는 보존)

In [21]:
def corona_count(data):
    # 코로나 기간에 해당되지 않는 데이터는 1, 코로나 기간은 0으로 설정하는 열 생성
    data['for_count'] = ((data['play_date'] <= '2020-03-22') | (data['play_date'] >= '2022-01-01')).astype(int)
    data['seat_count_covid'] = data.groupby('seat')['for_count'].transform('sum')
    data.drop(columns=['for_count'], inplace=True)

In [22]:
for c in conlist:
    corona_count(c) # 코로나 기간 보정한 count 열
    c['seat_count'] = c['seat'].map(c['seat'].value_counts()) # 코로나 기간 보정 없는 count

## 코로나 기간 공연별 예매율 변수 보정
- 거리두기 정책으로 인해 공연당 관람 가능 좌석이 줄어들어 전체적인 예매율이 낮게 형성됨
- **전체 기간의 평균 예매율**을 기준으로 코로나 기간의 예매율을 전반적으로 보정 처리 

In [23]:
# 코로나 기간의 띄어앉기 정책으로 인한 예매율 보정

for c in conlist:
    corona = c.sort_values('play_date')
    corona = c.drop_duplicates(subset='code', keep='first')
    corona.loc[(corona['play_date'] >= '2020-03-22') & (corona['play_date'] <= '2022-01-01'), 'advance_rate'] += (corona['advance_rate'].mean() - corona[(corona['play_date'] >= '2020-03-22') & (corona['play_date'] <= '2022-01-01')]['advance_rate'].mean())
    c['advance_rate'] = c['code'].map(corona.set_index('code')['advance_rate'])

* * *

## 좌석별 - 초대권 빈도, 할인율, 유료멤버십 비율 

In [25]:
# 초대권 빈도 수 / 결측치인 좌석은 0
for c in conlist:
    invitations = c['discount_type'] == '초대권'
    c['초대권빈도'] = c[invitations].groupby('seat')['discount_type'].transform('count')
    c['초대권빈도'].fillna(0, inplace=True)

In [26]:
# 좌석별 할인율 평균
for c in conlist:
    c['discount_percentage'].fillna(0, inplace=True)
    c['discount_percentage'] = c['discount_percentage'].astype(int)
    c['seat_discount_percentage'] = c.groupby('seat')['discount_percentage'].transform('mean')

In [27]:
# 좌석별 유료멤버의 비율, 기존 유료멤버십 카운트 열 삭제 - 대체적으로 1층 C블록 가운데 쯤이 높은거 확인
for c in conlist:
    c['seat_count'] = c.groupby('seat')['seat'].transform('count')
    c['seat_paid_member_rate'] = c['seat_paid_member_count'] / (c['seat_count_covid']+c['seat_count'])
    c.drop('seat_paid_member_count', axis=1, inplace=True)

# [데이터 시각화]
## 좌석 분포 시각화

In [9]:
cnt = 0
block_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'N': 8, 'M': 9}

In [ ]:
for c in conlist:
    cnt += 1
    plt.figure(figsize=(120,80))
    
    seat_c = c[~(c['seat'].str.contains('합창석') | c['seat'].str.contains('BOX'))]
    
    seat_viz = seat_c[['seat', '층', '블록', '열', '번호', 'seat_count_covid', 'seat_count']].drop_duplicates()
    seat_viz = seat_viz.sort_values(by=['층', '블록', '열', '번호']).reset_index(drop=True)
    
    vmin = seat_viz['seat_count_covid'].quantile(0.05)
    vmax = seat_viz['seat_count_covid'].quantile(0.95)
    
    seat_viz['블록'] = seat_viz['블록'].map(block_mapping)
    
    for floor in seat_viz['층'].unique():
        current_floor_df = seat_viz[seat_viz['층'] == floor]

        for block in current_floor_df['블록'].unique():
            block_df = current_floor_df[current_floor_df['블록'] == block]

            ax = plt.subplot(3, 9, (int(floor)-1)*9 + int(block))

            heatmap_data = block_df.pivot(index='열', columns='번호', values='seat_count_covid').sort_values(by='열', ascending=False)
            heatmap_data = heatmap_data[sorted(heatmap_data.columns, key=int)]
            sns.heatmap(heatmap_data, cmap="Blues", cbar=False, annot=True, fmt='g', vmin=vmin, vmax=vmax, ax=ax)

            ax.set_aspect(1)

            plt.title(f'Floor: {floor}, Block: {block}')
            plt.gca().invert_yaxis()  # y축 뒤집기

    plt.tight_layout()
    plt.savefig(f'{cnt}_viz.pdf', dpi=30)
    plt.savefig(f'{cnt}_viz.png', dpi=30)
    
    print(f'{cnt}번째 완료!')

1번째 완료!


KeyboardInterrupt: 

## 예매율 시각화

그 외 기타 EDA ...

* * *

### 데이터 내보내기

In [ ]:
symp.to_csv("콘서트홀_교향곡.csv", index=False)
clas.to_csv("콘서트홀_클래식.csv", index=False)
chor.to_csv("콘서트홀_합창.csv", index=False)
solo.to_csv("콘서트홀_독주.csv", index=False)
voca.to_csv("콘서트홀_성악.csv", index=False)
cham.to_csv("콘서트홀_실내악.csv", index=False) # 여기서부턴 좀 합쳐보든가 하자...
oper.to_csv("콘서트홀_오페라.csv", index=False)
conc.to_csv("콘서트홀_콘서트.csv", index=False)
comp.to_csv("콘서트홀_복합장르.csv", index=False) # 여기서부턴 버려도 될듯...
etc.to_csv("콘서트홀_기타.csv", index=False)
jazz.to_csv("콘서트홀_재즈.csv", index=False)
cros.to_csv("콘서트홀_크로스오버.csv", index=False)